In [1]:
import mysql.connector
import json
import uuid
import pandas as pd

In [2]:
# pip install mysql-connector-python


In [3]:
# Connection parameters
host = 'localhost'
user = 'root'
password = '1234'
database = 'stock_prediction'

In [4]:
data="../../crawlData/data_2212.json"
with open(data,'r') as file:
  data=json.load(file)

In [5]:
symbols=list(data.keys())
name_symbols=["Ngân hàng TMCP Á Châu","Ngân hàng TMCP Đầu tư và Phát triển Việt Nam","Ngân hàng TMCP Công Thương Việt Nam","Ngân hàng TMCP Xuất nhập khẩu Việt Nam","Ngân hàng TMCP Phát triển TP.HCM","Ngân hàng TMCP Quân Đội","Ngân hàng TMCP Hàng hải Việt Nam","Ngân hàng TMCP Phương Đông","Ngân hàng TMCP Sài Gòn – Hà Nội","Ngân hàng TMCP Đông Nam Á","Ngân hàng TMCP Sài Gòn Thương Tín","Ngân hàng TMCP Kỹ thương Việt Nam","Ngân hàng TMCP Tiên Phong","Ngân hàng TMCP Ngoại Thương Việt Nam","Ngân hàng TMCP Quốc tế Việt Nam","Ngân hàng TMCP Việt Nam Thịnh Vượng","Ngân hàng TMCP Quốc Dân","Ngân hàng TMCP Bắc Á"]

In [6]:
stocks=[]
for symbol in symbols:
  stocks.append(list(data[symbol].values()))
previous_close_prices=[]
for stock in stocks:
      previous_close_prices.append(eval(stock[0])["close"])


In [7]:
stock_ids=[]
for i in range (len(stocks)):
    stock_ids.append(str(uuid.uuid4()))

In [8]:
#create data insert to stocklist
data_stock_list_inserts=[]
for i in range(len(stocks)):
    data_stock_list_inserts.append((stock_ids[i],symbols[i],name_symbols[i],float(previous_close_prices[i].replace(',', ''))))

In [9]:
#convert dict to dataframe
from ast import literal_eval  
def convert_str_to_float(str_num):
    return float(str_num.replace(',', ''))
def convert_dict_to_df(data_dict):
    data = {key: literal_eval(value) for key, value in data_dict.items()}
    df = pd.DataFrame.from_dict(data, orient='index')
    df.index = pd.to_datetime(df.index, format='%d/%m/%Y')
    df = df.applymap(convert_str_to_float)
    df=df.sort_index(ascending=True)
    return df

In [10]:
data_historys=[]
for symbol in symbols:
  data_historys.append(data[symbol])
data_history_dfs=[]
for data_history in data_historys:
    data_history_dfs.append(convert_dict_to_df(data_history))

In [11]:
# Create a list to store the data inserts
data_history_inserts = []

for stockid, dataframe in zip(stock_ids, data_history_dfs):
    for index, row in dataframe.iterrows():
        
        date = row.name.date()  
        open_price = row['open']  
        high_price = row['high'] 
        low_price = row['low']  
        close_price = row['close']  
        volume = row['volume']  
        record = (stockid, str(date), open_price, high_price, low_price, close_price, volume)
       
        data_history_inserts.append(record)

In [12]:
# #insert to db
# try:
#     conn = mysql.connector.connect(
#         host=host,
#         user=user,
#         password=password,
#         database=database
#     )

#     if conn.is_connected():
#         print("Connected to MySQL")
        
#         cursor = conn.cursor()
#         # insert_stocklist_query = "INSERT INTO stocklist (stockid, symbol, company_name,previous_close_price) VALUES (%s, %s, %s,%s)"
#         # cursor.executemany(insert_stocklist_query, data_stock_list_inserts)

#         insert_stock_hist_query = "INSERT INTO stockhistory (stockid, date, open, high, low, close, volume) VALUES (%s, %s, %s, %s, %s, %s, %s)"
#         cursor.executemany(insert_stock_hist_query, data_history_inserts)
#         conn.commit()

#         cursor.close()
#         conn.close()

# except mysql.connector.Error as e:
#     print(f"Error: {e}")
# finally:
#     if conn.is_connected():
#         cursor.close()
#         conn.close()
#         print("MySQL connection closed")

In [13]:
insert_stock_hist_query = """
INSERT INTO stockhistory (stockid, date, open, high, low, close, volume) 
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [14]:
connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
)
cursor = connection.cursor()

In [15]:
from datetime import datetime
import ast  # Sử dụng ast.literal_eval để phân tích cú pháp an toàn hơn

for symbol, dates in data.items():
    # Bước 1: Lấy `stockid` của `symbol` từ bảng `stocklist`
    cursor.execute("SELECT stockid FROM stocklist WHERE symbol = %s", (symbol,))
    result = cursor.fetchone()

    if result:
        stockid = result[0]
        
        # Danh sách các bản ghi cần chèn cho mã chứng khoán hiện tại
        data_history_inserts = []
        
        # Bước 2: Chuẩn bị dữ liệu cho mỗi ngày
        for date_str, stock_data in dates.items():
            # Chuyển đổi ngày từ chuỗi thành định dạng datetime
            date = datetime.strptime(date_str, "%d/%m/%Y")
            
            # Chuyển chuỗi dictionary thành dictionary thực tế
            stock_data_dict = ast.literal_eval(stock_data)  # Sử dụng ast.literal_eval an toàn hơn eval
            
            # Lấy các giá trị từ dictionary và chuyển đổi chúng
            open_price = float(stock_data_dict['open'].replace(',', ''))
            high_price = float(stock_data_dict['high'].replace(',', ''))
            low_price = float(stock_data_dict['low'].replace(',', ''))
            close_price = float(stock_data_dict['close'].replace(',', ''))
            volume = int(stock_data_dict['volume'].replace(',', ''))
            
            # Thêm vào danh sách dữ liệu chèn
            data_history_inserts.append((stockid, date, open_price, high_price, low_price, close_price, volume))
        
        # Bước 3: Chèn dữ liệu vào stockhistory cho mã chứng khoán hiện tại
        cursor.executemany(insert_stock_hist_query, data_history_inserts)
        connection.commit()
        print(f"Dữ liệu đã được chèn thành công cho mã {symbol} vào bảng `stockhistory`.")
    else:
        print(f"Không tìm thấy `stockid` cho mã {symbol} trong bảng `stocklist`.")

# Đóng kết nối
cursor.close()
connection.close()

Dữ liệu đã được chèn thành công cho mã ACB vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã BID vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã CTG vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã EIB vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã HDB vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã MBB vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã MSB vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã OCB vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã SHB vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã SSB vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã STB vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã TCB vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã TPB vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã VCB vào bảng `stockhistory`.
Dữ liệu đã được chèn thành công cho mã VIB vào b

In [16]:
# import numpy as np
# try:
#     conn = mysql.connector.connect(
#         host=host,
#         user=user,
#         password=password,
#         database=database
#     )

#     if conn.is_connected():
#         print("Connected to MySQL")
#         cursor = conn.cursor()

#         # insert_stocklist_query = "INSERT INTO stocklist (stockid, symbol, company_name,previous_close_price) VALUES (%s, %s, %s,%s)"
#         # cursor.executemany(insert_stocklist_query, data_stock_list_inserts)
#         # Câu truy vấn chèn dữ liệu
#         insert_stock_hist_query = """
#         INSERT INTO stockhistory (stockid, date, open, high, low, close, volume) 
#         VALUES (%s, %s, %s, %s, %s, %s, %s)
#         """

#         # Chuyển đổi các giá trị trong data_history_inserts nếu cần
#         data_history_inserts = [
#             tuple(map(lambda x: float(x) if isinstance(x, np.float64) else x, row))
#             for row in data_history_inserts
#         ]

#         # Thực thi truy vấn
#         cursor.executemany(insert_stock_hist_query, data_history_inserts)
#         conn.commit()

#         cursor.close()
#         conn.close()
#         print("Dữ liệu đã được chèn thành công vào MySQL.")

# except mysql.connector.Error as e:
#     print(f"Error: {e}")
# finally:
#     if conn.is_connected():
#         cursor.close()
#         conn.close()
#         print("MySQL connection closed")